# 구조체와 클래스
## 객체지향 스위프트
## 옵셔널 체인

## 옵셔널 타입의 문제점

### 클래스나 구조체에 옵셔널 타입이 설정되었을 때에 대해 말해보자
- #### 스위프트에서 제공하는 옵셔널 타입은 nil이 할당될 수 있는 값을 말한다
- #### nil은 초기에 값 할당이 일어나지 않았을때 대입되지만, 값을 처리하는 과정에서 오류가 발생했을 때 대입되는 값이기도 한다

### 옵셔널 타입은 항상 nil 여부를 검사하여 정상적인 값이 저장된 것을 확인한 후에 사용하는 것이 안전하므로 if 구문을 통해 옵셔널 타입을 처리하는 경우가 많다
- #### 여기에서 옵셔널에 대한 이슈가 발생하는데, if 구문을 통해 값의 안전성 여부를 검사해야 한다는 것이다
    - #### 코드가 길어지기 때문이다
- #### 어차피 if 구문을 통해 값의 정상 여부를 검사해야 안전하게 사용할 수 있는 것이라면 굳이 옵셔널 타입을 사용할 필요도 없다는 것이 옵셔널을 다루어본 많은 사람들의 의견이다

### 문제는 클래스나 구조체가 옵셔널 타입과 관련되었을 때 발생한다.
- #### 클래스나 구조체 등의 객체는 내부적으로 프로퍼티나 메소드를 소유한다
    - #### 이때 클래스나 구조체의 인스턴스가 옵셔널 타입으로 선언될 경우 프로퍼티와 메소드를 호출하기 위해서는 매번 if 구문을 통해 옵셔널 인스턴스의 정상값 여부를 검사해야 한다

### 다음 코드를 보자

```
struct Human {
    var name: String?
    var man: Bool = true
}


var boy: Human? = Human(name:"홍길동", man: true)
```

- #### 인간을 나타내는 Human 구조체를 정의
- #### 이름을 저장할 name 프로퍼티, 남성 여부를 저장할 man 프로퍼티를 각각 추가하였다
- #### 이어서 이 구조체를 인스턴스로 생성하여 변수에 할당하되 옵셔널 타입의 변수 boy에 할당하고 있다

### 일단 옵셔널 타입으로 선언된 이상, 변수 boy를 사용하려면 옵셔널 타입에 대한 안전성 검사가 필요하다
- #### boy 인스턴스로부터 name 프로퍼티를 참조하려면 이 역시 옵셔널 타입이므로 다시 안전성 검사가 필요하다

### 다음과 같은 구문을 통해 name 프로퍼티를 참조해야 한다

```
if boy != nil {
    if boy!.name != nil {
        print("이름은\(boy!.name!)입니다")
    }
}
```

### 또는 옵셔널 비강제 해제 구문을 사용하여 다음과 같이 작성할 수도 있다

```
if let b = boy {
    if let name = b.name {
        print("이름은\(name)입니다")
    }
}
```

### 어느 방식이든 안전성을 담보하려면 if 구문의 처리를 피할 수 없다
- #### 만약 Human 구조체를 다른 구조체나 클래스가 프로퍼티로 사용하되, 이를 옵셔널 타입으로 설정한다면 name 프로퍼티를 참조하기 위한 코드는 훨씬 더 복잡해진다

```
struct Company {
    var ceo: Human?
    var companyName: String?
}
var startup: Company? = Company(ceo: Human(name:"최대표", man: false), companyName: "사랑해")
```

### Human 타입의 구조체 인스턴스를 옵셔널 타입으로 할당 받는 ceo 프로퍼티와, 
### 마찬가지로 옵셔널 타입이지만 문자열을 입력받는 companyName 프로퍼티가 선언된 Company 구조체이다

- #### 이 구조체 역시 옵셔널 타입으로 변수 startup 에 할당되었다

### 이제 변수 startup을 이용하여 ceo 프로퍼티인 name을 참조하는 과정을 단계별로 봐보자

### 가장 먼저 해야 할 일은 startup의 옵셔널 타입을 해제하는 것이다

```
if let company = startup {
    
}
```

- #### startup에 정상 값이 할당되어 있다면 위 구문의 실행 결과로 Company라는 상수에는 옵셔널이 해제된 Company 타입의 인스턴스가 들어있게 된다

### 다음으로 이 Company를 사용하여 ceo 프로퍼티의 옵셔널을 해제해야 한다

```
if let company = startup {
    if let ceo = company.ceo {
        
    }
}
```

### 마지막으로 ceo 프로퍼티의 하위 프로퍼티인 name 역시 옵셔널 타입이므로 해제 과정을 거쳐야 회사의 대표이사 이름을 참조해낼 수 있다

```
if let company = startup {
    if let ceo = company.ceo {
        if let name = ceo.name {
            print("대표이사의 이름은 \(name)입니다")
        }
    }
}
```

### 이러한 과정을 피하고 코드를 더욱 간결하게 작성하기 위해 강제 해제 연산자!를 사용하여 if 구문 없이 인스턴스의 옵셔널 타입을 해제할 수도 있다
- #### 그러나 이것은 언제든 발생할 가능성이 있는 런타임 오류라는 위험요소를 내포하고 있으므로 추천할 만한 방법은 아니다
- #### 강제 해제 연산자를 사용하는 어느 중간 과정이 하나라도 nil을 포함하고 있다면 여지없이 런타입 오류가 발생하기 때문이다

```
if let name = startup!.ceo!.name {
    print("대표이상의 이름은 \(name)입니다")
}
```

### 코코아 터치 프레임워크에서는 이처럼 옵셔널 타입을 중첩해서 사용해야 하는 경우가 많다
- #### 단계적으로 객체를 만들어 다음 단계로 접근해야 하는데, 이때 각 단계별 결과물이 옵셔널 타입인 경우가 대부분이기 때문이다

### 이처럼 옵셔널 타입이 중첩되어 있을 때 매번 if 구문을 중첩해서 작성하는 것은 코드를 작성해야 하는 입장에서 상당한 부담이 된다
- #### 작성한 코드를 쉽게 파악하기도 어려울뿐더러 실제로 구현해야 하는 논리 흐름에 집중하기보다는 객체의 오류 가능성을 차단하고자 몇 배나 되는 코드를 작성해야 하는 것도 문제다
- #### 이러한 옵셔널의 치명적인 단점을 극복하고 복잡한 코드를 간단하게 줄여주는 방법으로 도입된 것이 바로 옵셔널 체인이다